In [1]:
#tomoeのaz1[deg]とaz2[deg]をmuのstartpoint,endpointに合わせる
#向き
import pandas as pd
import numpy as np
import astropy.wcs

df1 = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e4.csv")
df2 = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e4.csv")

az1_tomoe = (df1["az1[deg]"]).values
el1_tomoe = (df1["el1[deg]"]).values
az2_tomoe = (df1["az2[deg]"]).values
el2_tomoe = (df1["el2[deg]"]).values
index = np.array(range(2000))
for (i, j, k, l, m) in zip(index, az1_tomoe, az2_tomoe,\
                          el1_tomoe, el2_tomoe):
    if j > k:
        az1_tomoe[i] = k
        az2_tomoe[i] = j
        el1_tomoe[i] = m
        el2_tomoe[i] = l
df2["az1[deg]"] = az1_tomoe
df2["az2[deg]"] = az2_tomoe
df2["el1[deg]"] = el1_tomoe
df2["el2[deg]"] = el2_tomoe
df2.to_csv("simultaneous_event_candidates_table_MU+Tomo-e4.csv")
print "Complete!"

Complete!


In [2]:
#MUとTomo-eの線分が被っている割合
import pandas as pd
import numpy as np
import astropy.wcs

df1 = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e4.csv")

#muのベクトルの直線の式をだす
az1_mu = (df1["azimuth_start_at_kiso"]).values
el1_mu = (df1["elevation_start_at_kiso"]).values
az2_mu = (df1["azimuth_end_at_kiso"]).values
el2_mu = (df1["elevation_end_at_kiso"]).values
slope_mu = (el2_mu - el1_mu)/(az2_mu - az1_mu)
b_mu = el1_mu + slope_mu*az1_mu*(-1)
slope = (-1)*1/slope_mu
#tomoeのベクトルの直線の式をだす
az1_tomoe = (df1["az1[deg]"]).values
el1_tomoe = (df1["el1[deg]"]).values
az2_tomoe = (df1["az2[deg]"]).values
el2_tomoe = (df1["el2[deg]"]).values
slope_tomoe = (el2_tomoe - el1_tomoe)/(az2_tomoe - az1_tomoe)
b_tomoe = el1_tomoe + slope_tomoe*az1_tomoe*(-1)

#tomoeのベクトル(az, el)を伸ばす
#azimuthのレンジは[226 : 242]
#elevationのレンジは[23 : 38]
az3_tomoe = np.full(az1_tomoe.shape, 226.0)
az4_tomoe = np.full(az1_tomoe.shape, 242.0)
el3_tomoe = slope_tomoe*az3_tomoe + b_tomoe
el4_tomoe = slope_tomoe*az4_tomoe + b_tomoe
index = np.array(range(2000))
for (i, j) in zip(el3_tomoe, index):
    if i > 38.0:
        el3_tomoe[j] = 38.0
        az3_tomoe[j] = (38.0 - b_tomoe[j])/slope_tomoe[j]
    elif i < 23.0:
        el3_tomoe[j] = 23.0
        az3_tomoe[j] = (23.0 - b_tomoe[j])/slope_tomoe[j]

for (i, j) in zip(el4_tomoe, index):
    if i > 38.0:
        el4_tomoe[j] = 38.0
        az4_tomoe[j] = (38.0 - b_tomoe[j])/slope_tomoe[j]
    elif i < 23.0:
        el4_tomoe[j] = 23.0
        az4_tomoe[j] = (23.0 - b_tomoe[j])/slope_tomoe[j]        
df1["az3[deg]_tomoe"] = pd.DataFrame(az3_tomoe)
df1["el3[deg]_tomoe"] = pd.DataFrame(el3_tomoe)
df1["az4[deg]_tomoe"] = pd.DataFrame(az4_tomoe)
df1["el4[deg]_tomoe"] = pd.DataFrame(el4_tomoe)

#muとtomoeの角両端の点から直線との交点を求める
#part1 muの両端の点からtomoeのベクトルとの交点を求める
b1_mu = el1_mu + slope*az1_mu*(-1)
b2_mu = el2_mu + slope*az2_mu*(-1)
az1_intersection_tomoe = (b1_mu - b_tomoe)/(slope_tomoe - slope)
el1_intersection_tomoe = (slope_tomoe*b1_mu - slope*b_tomoe)/(slope_tomoe - slope)
az2_intersection_tomoe = (b2_mu - b_tomoe)/(slope_tomoe - slope)
el2_intersection_tomoe = (slope_tomoe*b2_mu - slope*b_tomoe)/(slope_tomoe - slope)

#part2 tomoeの両端の点からtomoeのベクトルとの交点を求める
b1_tomoe = el1_tomoe + slope*az1_tomoe*(-1)
b2_tomoe = el2_tomoe + slope*az2_tomoe*(-1)
az1_intersection_mu = (b1_tomoe - b_mu)/(slope_mu - slope)
el1_intersection_mu = (slope_mu*b1_tomoe - slope*b_mu)/(slope_mu - slope)
az2_intersection_mu = (b2_tomoe - b_mu)/(slope_mu - slope)
el2_intersection_mu = (slope_mu*b2_tomoe - slope*b_mu)/(slope_mu - slope)

df1["az1_intersection_tomoe"] = pd.DataFrame(az1_intersection_tomoe)
df1["el1_intersection_tomoe"] = pd.DataFrame(el1_intersection_tomoe)
df1["az2_intersection_tomoe"] = pd.DataFrame(az2_intersection_tomoe)
df1["el2_intersection_tomoe"] = pd.DataFrame(el2_intersection_tomoe)
df1["az1_intersection_mu"] = pd.DataFrame(az1_intersection_mu)
df1["el1_intersection_mu"] = pd.DataFrame(el1_intersection_mu)
df1["az2_intersection_mu"] = pd.DataFrame(az2_intersection_mu)
df1["el2_intersection_mu"] = pd.DataFrame(el2_intersection_mu)
#tomoeベクトルの長さを求める
lx1_tomoe = az1_tomoe - az2_tomoe #x
ly1_tomoe = el1_tomoe - el2_tomoe #y
lx1_tomoe = np.power(lx1_tomoe, 2) #x^2
ly1_tomoe = np.power(ly1_tomoe, 2) #y^2
l1_tomoe = lx1_tomoe + ly1_tomoe #x^2+y^2
l1_tomoe = np.sqrt(l1_tomoe) 

#それぞれの交点でパターンに分ける
#ノート参照(全て合致="1"、一部合致="2&3",全て否合致="0")
#az1_intersection_mu, az2_intersection_mu
#az1_intersection_tomoe, az2_intersection_tomoe
group = np.full(az1_mu.shape, 0, dtype="float")
for (i, j, k, l, m) in zip(index, az1_tomoe, az2_tomoe, \
                  az1_intersection_tomoe, az2_intersection_tomoe):
    if l < j and k < m:#group "1"
        group[i] = 1.0 
    elif l < j and m < k and j < m < k:#group "2"
        group[i] = 2.0
    elif j < l and k < m and j < l < k:#group "3"
        group[i] = 3.0
    elif l < j and m < k:#group "0"
        group[i] = 0.0
##group "1"のものに対して検出領域が被っている比率を求める
#tomoeのベクトル全体の長さを求める
for (i, j) in zip(index, group):
    if group[i] == 2.0:
        lx2_tomoe = az1_tomoe - az2_intersection_tomoe #x
        ly2_tomoe = el1_tomoe - el2_intersection_tomoe #y
        lx2_tomoe = np.power(lx2_tomoe, 2) #x^2
        ly2_tomoe = np.power(ly2_tomoe, 2) #y^2
        l2_tomoe = lx2_tomoe + ly2_tomoe #x^2+y^2
        l2_tomoe = np.sqrt(l2_tomoe)
        ratio1 = l2_tomoe/l1_tomoe
        group[i] = ratio1[i]
    if group[i] == 3.0:
        lx3_tomoe = az1_intersection_tomoe - az2_tomoe #x
        ly3_tomoe = el1_intersection_tomoe - el2_tomoe #y
        lx3_tomoe = np.power(lx1_tomoe, 2) #x^2
        ly3_tomoe = np.power(ly1_tomoe, 2) #y^2
        l3_tomoe = lx3_tomoe + ly3_tomoe #x^2+y^2
        l3_tomoe = np.sqrt(l3_tomoe)
        ratio2 = l3_tomoe/l1_tomoe
        group[i] = ratio2[i]

percentage = 100*group
columns = ["percentage"]
percentage = pd.DataFrame(data = percentage, columns = columns)
df1 = pd.concat([df1, percentage], axis=1)
df1.to_csv("simultaneous_event_candidates_table_MU+Tomo-e5.csv")
print "Complete!"

Complete!


In [5]:
#orbit_dataと照合して一つのファイルにまとめる、突入角の計算 , ノート参照
import pandas as pd
import numpy as np

####これは書き直す必要あり
###動くには動くけどとても面倒臭いことをしてる



df1 = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e5.csv")
df2 = pd.read_csv("MU_orbit_data_20181023.csv")

day_1 = (df1["day_jst_mu"]).values
hour_1 = (df1["hour_jst_mu"]).values
minute_1 = (df1["minute_jst_mu"]).values
second_1 = (df1["second_jst_mu"]).values
second_1 = np.round(second_1, 4)
time_1 = np.vstack((day_1, hour_1, minute_1, second_1))
time_1 = time_1.transpose()

day_2 = (df2["DD"]).values
hour_2 = (df2["hh"]).values
minute_2 = (df2["mm"]).values
second_2 = (df2["ss.ssssss"]).values
second_2 = np.round(second_2, 4)

#時刻をjstに統一する
index_0 = np.array(range(4200))
index_1 = np.array(range(1870))#1870
index_2 = np.array(range(4017))#4017
hour_2 = hour_2 + 9.
for (i, j, k) in zip(index_0, hour_2, day_2):
    if hour_2[i] >= 24:
        hour_2[i] = hour_2[i] - 24.
        day_2[i] = day_2[i] + 1.
time_2 = np.vstack((day_2, hour_2, minute_2, second_2))
time_2 = time_2.transpose()

index_number = np.arange(1).reshape(1) #indexナンバーを格納する配列
for i in index_1:
    for j in index_2:
        if time_1[i,0] == time_2[j,0] and time_1[i,1] == time_2[j,1] and \
        time_1[i,2] == time_2[j,2] and time_1[i,3] == time_2[j,3]:
            index_number = np.vstack((index_number, j))
index_number = np.delete(index_number,0, 0)
index_number = np.reshape(index_number,(1870))#1870

concat_table = np.array(range(54)) #df1と同じものを格納する配列
a_df2 = df2.values
for i in index_number:
    elements = a_df2[i,:]
    concat_table = np.vstack((concat_table, elements))

concat_table = np.delete(concat_table, 0, 0)
columns = df2.columns.values
df3 = pd.DataFrame(data = concat_table, columns = columns)
df4 = pd.concat([df1, df3], axis=1)
#####################################################################################
###############################################  突入角の計算 , ノート参照
beginning_height = (df4["Beginning_height[km]"]).values
end_height = (df4["End height [km]"]).values
az_start = (df4["Azimuth of meteor start point"]).values
zenith_start = (df4["Zenith of meteor start point"]).values
az_end = (df4["Azimuth of meteor end point"]).values
zenith_end = (df4["Zenith of meteor end point"]).values
########## start point
z_start = beginning_height
r_start = z_start/np.cos(np.deg2rad(zenith_start)) #z = height[km]
x_start = r_start*np.sin(np.deg2rad(zenith_start))*np.cos(np.deg2rad(az_start))
y_start = r_start*np.sin(np.deg2rad(zenith_start))*np.sin(np.deg2rad(az_start))
########## end point
z_end = end_height
r_end = z_end/np.cos(np.deg2rad(zenith_end)) #z = height[km]
x_end = r_end*np.sin(np.deg2rad(zenith_end))*np.cos(np.deg2rad(az_end))
y_end = r_end*np.sin(np.deg2rad(zenith_end))*np.sin(np.deg2rad(az_end))
##################  ベクトル計算
z = z_end - z_start
x = x_end - x_start
y = y_end - y_start
##################　もう一度極座標系でベクトルの天頂角(突入角)を出す
lx = np.power(x, 2) #x^2
ly = np.power(y, 2) #y^2
lz = np.power(z, 2) #z^2
l = lx + ly + lz #x^2+y^2+z^2
l = np.sqrt(l)
theta = np.rad2deg(np.arccos(abs(z)/l))
table = np.vstack((x_start,y_start,z_start,x_end,y_end,z_end,theta))
table = table.transpose()
columns = ["x_start[km]","y_start[km]","z_start[km]","x_end[km]","y_end[km]","z_end[km]","incidence_angle[deg]"]
df_angle = pd.DataFrame(data = table, columns = columns)
df5 = pd.concat([df4, df_angle], axis=1)
df5.to_csv("simultaneous_event_candidates_table_MU+Tomo-e6.csv")
print "Complete!"

Complete!
